In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

**Note**: Since processing all the text takes around ~5-10 minutes, in the cell below we are taking only the first tenth of each text. If you want to experiment, feel free to change the following code in the next cell:

```python
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])
```
to 

```python
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)
```

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not 
    # recognize: the double dash '--'. Better get rid of it now!
    text = re.sub(r'--', ' ', text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic.
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row{}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [6]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row0
Processing row50
Processing row100
Processing row150
Processing row200
Processing row250
Processing row300
Processing row350
Processing row400
Processing row450
Processing row500
Processing row550
Processing row600
Processing row650
Processing row700
Processing row750
Processing row800
Processing row850
Processing row900
Processing row950
Processing row1000
Processing row1050
Processing row1100
Processing row1150
Processing row1200
Processing row1250
Processing row1300
Processing row1350
Processing row1400
Processing row1450
Processing row1500
Processing row1550
Processing row1600
Processing row1650
Processing row1700
Processing row1750
Processing row1800
Processing row1850
Processing row1900
Processing row1950
Processing row2000
Processing row2050
Processing row2100
Processing row2150
Processing row2200
Processing row2250
Processing row2300
Processing row2350
Processing row2400
Processing row2450
Processing row2500
Processing row2550
Processing row2600
Processing row26

,size,walter,with,orange,plainly,carteret,shin,cur,estate,poetry,...,force,claws,care,rejoice,laughing,interesting,sir,roughly,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [7]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence', 'text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9871473354231975

Test set score: 0.8923872180451128


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', solver='lbfgs') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.9579937304075236

Test set score: 0.9168233082706767


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8731203007518797


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data. Remember that for computation time concerns, we only took the first tenth of the Alice text. Emma is pretty long. **So in order to get comparable length texts, we take the first sixtieth of Emma**. Again, if you want to experiment, you can take the whole texts of each.

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row0
Processing row50
Processing row100
Processing row150
Processing row200
Processing row250
Processing row300
Processing row350
Processing row400
Processing row450
Processing row500
Processing row550
Processing row600
Processing row650
Processing row700
Processing row750
Processing row800
Processing row850
Processing row900
Processing row950
Processing row1000
Processing row1050
Processing row1100
Processing row1150
Processing row1200
Processing row1250
Processing row1300
Processing row1350
Processing row1400
Processing row1450
Processing row1500
Processing row1550
Processing row1600
Processing row1650
Processing row1700
Processing row1750
Processing row1800
Processing row1850
Processing row1900
Processing row1950
Processing row2000
Processing row2050
Processing row2100
Processing row2150
Processing row2200
Processing row2250
Processing row2300
Processing row2350
Processing row2400
Processing row2450
Processing row2500
Processing row2550
Processing row2600
Processing row26

In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence', 'text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.872554950594878


col_0,Austen,Carroll
row_0,,
Austen,8347,558
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

# Challenge 0:

In [15]:
# First, let's run an SVM model.
# Import required packages.
from sklearn.svm import LinearSVC

svc = LinearSVC()
svc.fit(X_train, y_train)
print(svc)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Training set score: 0.9843260188087775

Test set score: 0.8980263157894737


In [16]:
# Let's calculate word frequencies.
def word_frequencies(text, include_stop=True):
    # Build a list of words.
    # Strip out punctuation and, optionally, stop words.
    words = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            words.append(token.text)
            
    # Build and return a Counter object containing word counts.
    return Counter(words)

# The most frequent words:
emma_freq = word_frequencies(emma_doc, include_stop=False).most_common(10)
print('Emma:', emma_freq)
alice_freq = word_frequencies(alice_doc, include_stop=False).most_common(10)
print('Alice:', alice_freq)
persuasion_freq = word_frequencies(persuasion_doc, include_stop=False).most_common(10)
print('Persuasion:', persuasion_freq)

Emma: [('I', 3187), ('Mr.', 1152), ("'s", 932), ('Emma', 864), ('Mrs.', 699), ('Miss', 595), ('She', 570), ('Harriet', 506), ('said', 484), ('He', 443)]
Alice: [('I', 540), ('said', 456), ('Alice', 395), ("n't", 216), ("'s", 192), ('little', 125), ('The', 103), ('know', 87), ('like', 84), ('went', 83)]
Persuasion: [('I', 1121), ('Anne', 497), ("'s", 485), ('She', 327), ('Captain', 297), ('Mrs', 291), ('Elliot', 288), ('Mr', 256), ('He', 225), ('Wentworth', 218)]


In [17]:
# Pull out just the text from our frequency lists.
alice_common = [pair[0] for pair in alice_freq]
persuasion_common = [pair[0] for pair in persuasion_freq]
emma_common = [pair[0] for pair in emma_freq]

# Use sets to find the unique values in each top ten.
print('Unique to Alice:', set(alice_common) - set(persuasion_common) - set(emma_common))
print('Unique to Persuasion:', set(persuasion_common) - set(alice_common) - set(persuasion_common))
print('Unique to Emma:', set(emma_common) - set(alice_common) - set(persuasion_common))

Unique to Alice: {'know', 'Alice', 'went', "n't", 'The', 'like', 'little'}
Unique to Persuasion: set()
Unique to Emma: {'Mr.', 'Emma', 'Miss', 'Harriet', 'Mrs.'}


Apparently there are no unique words in Persuasion, so let's run it again with 25 words each.

In [18]:
# The most frequent words:
emma_freq = word_frequencies(emma_doc, include_stop=False).most_common(25)
print('Emma:', emma_freq)
alice_freq = word_frequencies(alice_doc, include_stop=False).most_common(25)
print('Alice:', alice_freq)
persuasion_freq = word_frequencies(persuasion_doc, include_stop=False).most_common(25)
print('Persuasion:', persuasion_freq)

Emma: [('I', 3187), ('Mr.', 1152), ("'s", 932), ('Emma', 864), ('Mrs.', 699), ('Miss', 595), ('She', 570), ('Harriet', 506), ('said', 484), ('He', 443), ('Weston', 438), ('It', 400), ('thing', 399), ('Knightley', 389), ('Elton', 387), ('think', 381), ('The', 358), ('little', 356), ('good', 340), ('know', 337), ('Woodhouse', 314), ('You', 307), ('Jane', 300), ('But', 293), ('time', 273)]
Alice: [('I', 540), ('said', 456), ('Alice', 395), ("n't", 216), ("'s", 192), ('little', 125), ('The', 103), ('know', 87), ('like', 84), ('went', 83), ('thought', 74), ('Queen', 73), ('time', 68), ('And', 67), ('It', 64), ('King', 61), ('began', 58), ('Turtle', 58), ("'m", 57), ('way', 56), ('Hatter', 55), ('Mock', 55), ('Gryphon', 55), ("'ll", 53), ('You', 51)]
Persuasion: [('I', 1121), ('Anne', 497), ("'s", 485), ('She', 327), ('Captain', 297), ('Mrs', 291), ('Elliot', 288), ('Mr', 256), ('He', 225), ('Wentworth', 218), ('The', 209), ('Lady', 191), ('good', 181), ('It', 181), ('little', 175), ('said',

In [19]:
# Pull out just the text from our frequency lists.
alice_common = [pair[0] for pair in alice_freq]
persuasion_common = [pair[0] for pair in persuasion_freq]
emma_common = [pair[0] for pair in emma_freq]

# Use sets to find the unique values in each top ten.
print('Unique to Alice:', set(alice_common) - set(persuasion_common) - set(emma_common))
print('Unique to Persuasion:', set(persuasion_common) - set(alice_common) - set(persuasion_common))
print('Unique to Emma:', set(emma_common) - set(alice_common) - set(persuasion_common))

Unique to Alice: {'began', 'Alice', 'went', "n't", 'way', "'ll", "'m", 'Hatter', 'King', 'Gryphon', 'Queen', 'And', 'like', 'thought', 'Mock', 'Turtle'}
Unique to Persuasion: set()
Unique to Emma: {'Elton', 'thing', 'Woodhouse', 'Mr.', 'Knightley', 'But', 'Weston', 'Emma', 'Miss', 'Harriet', 'Jane', 'Mrs.'}


In [20]:
# Utility function to calculate how frequently lemmas appear in the text.
def lemma_frequencies(text, include_stop=True):
    
    # Build a list of lemmas.
    # Strip out punctuation and, optionally, stop words.
    lemmas = []
    for token in text:
        if not token.is_punct and (not token.is_stop or include_stop):
            lemmas.append(token.lemma_)
            
    # Build and return a Counter object containing word counts.
    return Counter(lemmas)

# Instantiate our list of most common lemmas.
alice_lemma_freq = lemma_frequencies(alice_doc, include_stop=False).most_common(10)
persuasion_lemma_freq = lemma_frequencies(persuasion_doc, include_stop=False).most_common(10)
emma_lemma_freq = lemma_frequencies(emma_doc, include_stop=False).most_common(10)
print('\nAlice:', alice_lemma_freq)
print('Persuasion:', persuasion_lemma_freq)
print('Emma:', emma_lemma_freq)

# Again, identify the lemmas common to one text but not the others.
alice_lemma_common = [pair[0] for pair in alice_lemma_freq]
persuasion_lemma_common = [pair[0] for pair in persuasion_lemma_freq] 
emma_lemma_common = [pair[0] for pair in emma_lemma_freq]

print('Unique to Alice:', set(alice_lemma_common) - set(persuasion_lemma_common) - set(emma_lemma_common))
print('Unique to Persuasion:', set(persuasion_lemma_common) - set(alice_lemma_common) - set(persuasion_lemma_common))
print('Unique to Emma:', set(emma_lemma_common) - set(alice_lemma_common) - set(persuasion_lemma_common))


Alice: [('-PRON-', 765), ('say', 479), ('alice', 397), ('be', 258), ('not', 232), ('think', 134), ('go', 133), ('little', 127), ('the', 110), ('know', 107)]
Persuasion: [('-PRON-', 2243), ('anne', 497), ("'s", 466), ('captain', 303), ('elliot', 295), ('mrs', 291), ('good', 289), ('know', 258), ('think', 257), ('mr', 256)]
Emma: [('-PRON-', 5467), ('mr.', 1152), ("'s", 886), ('emma', 864), ('mrs.', 699), ('think', 660), ('miss', 613), ('say', 565), ('good', 547), ('know', 541)]
Unique to Alice: {'be', 'not', 'the', 'alice', 'go', 'little'}
Unique to Persuasion: set()
Unique to Emma: {'mrs.', 'mr.', 'emma', 'miss'}


In [21]:
# Let's see how many sentences are in each book.
sents_alice = list(alice_doc.sents)
sents_persuasion = list(persuasion_doc.sents)
sents_emma = list(emma_doc.sents)

print("Alice in Wonderland has {} sentences.".format(len(sents_alice)))
print("Persuasion has {} sentences.".format(len(sents_persuasion)))
print("Emma has {} sentences.".format(len(sents_emma)))


Alice in Wonderland has 1669 sentences.
Persuasion has 3649 sentences.
Emma has 8905 sentences.


In [22]:
# Let's copy our word_counts data frame as a form of version control.
word_counts2 = word_counts

# Add a column for the word counts in each sentence to the data frame.
word_counts2['sent_length'] = word_counts2.text_sentence.map(lambda x: len(x))

# Let's create a count for parts of speech.
# Adverbs in each sentence.
sentences2 = word_counts2.text_sentence
adv_count = []
for sent in sentences2:
    advs = 0
    for token in sent:
        if token.pos_ == 'ADV':
            advs +=1
    adv_count.append(advs)
            
# Add adverbs column to data frame.
word_counts2['adv_count'] = adv_count


# Verbs in each sentence.
verb_count = []
for sent in sentences2:
    verb = 0
    for token in sent:
        if token.pos_ == 'VERB':
            verb +=1
    verb_count.append(verb)
    
# Add verbs column to data frame.
word_counts2['verb_count'] = verb_count


# Nouns in each sentence:
noun_count = []
for sent in sentences2:
    noun = 0
    for token in sent:
        if token.pos_ == 'NOUN':
            noun +=1
    noun_count.append(noun)
    
# Add nouns column to data frame.
word_counts2['noun_count'] = noun_count

# Punctuation marks in each sentence.
punct_count = []
for sent in sentences2:
    punct = 0
    for token in sent:
        if token.pos_ == 'PUNCT':
            punct +=1
    punct_count.append(punct)
    
# Add punctuation column to data frame.
word_counts2['punct_count'] = punct_count

word_counts2.head()

,size,walter,with,orange,plainly,carteret,shin,cur,estate,poetry,...,interesting,sir,roughly,text_sentence,text_source,sent_length,adv_count,verb_count,noun_count,punct_count
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,3,13,12,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,7,11,8,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,6,5,2,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Oh, dear, !)",Carroll,3,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(I, shall, be, late, !, ')",Carroll,6,0,2,0,2


In [23]:
# Let's set up our model again.
y = word_counts2['text_source']
X = np.array(word_counts2.drop(['text_sentence', 'text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression(solver='lbfgs', max_iter=5000)
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score: ', lr.score(X_train, y_train))
print('\nTest set score: ', lr.score(X_test, y_test))

(3190, 3067) (3190,)
Training set score:  0.9626959247648903

Test set score:  0.9210526315789473


In [24]:
# Let's go back and re-try SVM with the new features.

svm = LinearSVC()
train = svm.fit(X_train, y_train)
print('Training set score: ', svm.score(X_train, y_train))
print('\nTest set score: ', svm.score(X_test, y_test))

Training set score:  0.9855799373040752

Test set score:  0.9083646616541353


/Users/danmchenry/miniconda3/lib/python3.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
# Let's see this in crosstab.
svm_predicted = lr.predict(X_test)
pd.crosstab(y_test, svm_predicted)

col_0,Austen,Carroll
text_source,,
Austen,1438,34
Carroll,134,522


This model seems extremely good at knowing when a sentence is written by Austen, but not very good at identifying Carroll. It is only due to class imbalance that our results appear to be so good (if the classes were equally balanced, our model would be much less effective). Let's try Challenge 1.

# Challenge 1:

In [26]:
# Let's load another book. Let's see the different vocabulary in Whitman's *Leaves of Grass*.
leaves = gutenberg.raw('whitman-leaves.txt')
print('\nRaw:\n', leaves[0:100])


Raw:
 [Leaves of Grass by Walt Whitman 1855]


Come, said my soul,
Such verses for my Body let us write, (


In [27]:
# Let's use our cleaner function to clean the text.
leaves = text_cleaner(leaves[:int(len(leaves)/10)])

# Parse the clean data.
leaves_doc = nlp(leaves)

In [28]:
# Preview cleaned text.
print(leaves[:1000])

Come, said my soul, Such verses for my Body let us write, (for we are one,) That should I after return, Or, long, long hence, in other spheres, There to some group of mates the chants resuming, (Tallying Earth's soil, trees, winds, tumultuous waves,) Ever with pleas'd smile I may keep on, Ever and ever yet the verses owning as, first, I here and now Signing for Soul and Body, set to them my name, Walt Whitman } One's-Self I Sing One's-self I sing, a simple separate person, Yet utter the word Democratic, the word En-Masse. Of physiology from top to toe I sing, Not physiognomy alone nor brain alone is worthy for the Muse, I say the Form complete is worthier far, The Female equally with the Male I sing. Of Life immense in passion, pulse, and power, Cheerful, for freest action form'd under the laws divine, The Modern Man I sing. } As I Ponder'd in Silence As I ponder'd in silence, Returning upon my poems, considering, lingering long, A Phantom arose before me with distrustful aspect, Terri

In [29]:
# Group by sentences.
leaves_sents = [[sent, 'Whitman'] for sent in leaves_doc.sents]

leaves_sents = leaves_sents[0:len(alice_sents)]

In [30]:
# Try a new Bag of Words for Leaves word counts.
leaves_sentences = pd.DataFrame(leaves_sents)
leaves_bow = bow_features(leaves_sentences, common_words)

print('done')

Processing row0
Processing row50
Processing row100
Processing row150
Processing row200
Processing row250
Processing row300
Processing row350
Processing row400
Processing row450
Processing row500
Processing row550
Processing row600
done


In [31]:
# Bag of words function.
def bow_counts(bow_df):
    counts = []
    # Count number of each feature per sentence.
    for sent in bow_df.text_sentence:
        len_sent = len(sent)
        advs = 0
        verbs = 0
        nouns = 0
        punc = 0
        for token in sent:
            if token.pos_ == 'ADV':
                advs += 1
            elif token.pos_ == 'VERB':
                verbs += 1
            elif token.pos_ == 'NOUN':
                nouns += 1
            elif token.pos_ == 'PUNCT':
                punc += 1
        # Append counts to list.
        counts.append([len_sent, advs, verbs, nouns, punc])
    # Combine original bow_features df with counts.
    df = pd.concat([bow_df, pd.DataFrame(counts, columns=['sent_length',
                                                          'adv_count',
                                                          'verb_count',
                                                          'noun_count',
                                                          'punc_count'])], axis=1)
    return df
    

In [32]:
leaves_bow.head()

,size,walter,with,orange,plainly,carteret,shin,cur,estate,poetry,...,force,claws,care,rejoice,laughing,interesting,sir,roughly,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Come, ,, said, my, soul, ,, Such, verses, for...",Whitman
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(That, should, I, after, return, ,, Or, ,, lon...",Whitman
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Ever, with, pleas'd, smile, I, may, keep, on,...",Whitman
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, here, and, now, Signing, for, Soul, and, B...",Whitman
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,(One's),Whitman


In [34]:
# Group into sentences.
leaves_sents = [[sent, "Whitman"] for sent in leaves_doc.sents]

In [35]:
# Build a new Bag of Words data frame for Leaves word counts.
# We'll use the same common words from Alice and Persuasion.
leaves_sentences = pd.DataFrame(leaves_sents)
leaves_bow = bow_features(leaves_sentences, common_words)

print('done')

Processing row0
Processing row50
Processing row100
Processing row150
Processing row200
Processing row250
Processing row300
Processing row350
Processing row400
Processing row450
Processing row500
Processing row550
Processing row600
done


In [36]:
# Let's set up our model again.
y = word_counts2['text_source']
X = np.array(word_counts2.drop(['text_sentence', 'text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression(solver='lbfgs', max_iter=5000)
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score: ', lr.score(X_train, y_train))
print('\nTest set score: ', lr.score(X_test, y_test))

(3190, 3067) (3190,)
Training set score:  0.9626959247648903

Test set score:  0.9210526315789473
